In [ ]:
import QuantLib as ql
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter
from datetime import date

today = date.today()
start_date = date(2000, 1, 1)
date_dates = []
dfs = []

# yfinanceでTreasuryイールドを取得
tickers = {
    "^IRX": ql.Period(3, ql.Months),   # 13週T-Bill（約3ヶ月）
    "^FVX": ql.Period(5, ql.Years),    # 5年Treasury Note
    "^TNX": ql.Period(10, ql.Years),   # 10年Treasury Note
    "^TYX": ql.Period(30, ql.Years),   # 30年Treasury Bond
}

# カレンダーと日数計算規則
calendar = ql.UnitedStates(ql.UnitedStates.GovernmentBond)
day_count = ql.ActualActual(ql.ActualActual.Bond)

treasury = yf.Tickers(list(tickers.keys()))
hist = treasury.history(start=start_date, end=today)
for date_timestamp in hist.index:
    date_datetime = date_timestamp.to_pydatetime()
    eval_date = date_datetime.date() # 任意の日付に変更可能
    ql.Settings.instance().evaluationDate = ql.Date(eval_date.day, eval_date.month, eval_date.year)

    data = []
    for ticker, tenor in tickers.items():
        if ticker in hist["Close"] and not hist["Close"][ticker].isna().all():
            yield_rate = hist["Close"][ticker].loc[date_timestamp] / 100
            data.append((tenor, yield_rate))
        else:
            print(f"データ取得失敗: {ticker}")

    # カーブ構築用のヘルパー
    quotes = []
    helpers = []
    for tenor, rate in data:
        quote = ql.SimpleQuote(rate)
        quotes.append(quote)
        helper = ql.DepositRateHelper(
            ql.QuoteHandle(quote),
            tenor,
            2,  # fixing days
            calendar,
            ql.ModifiedFollowing,
            True,
            day_count
        )
        helpers.append(helper)

    # イールドカーブの構築
    curve = ql.PiecewiseLogCubicDiscount(
        0,  # settlement days
        calendar,
        helpers,
        day_count
    )
    curve.enableExtrapolation()

    # カーブからイールドを取得（1～30年）
    dates = [ql.Date(eval_date.day, eval_date.month, eval_date.year) + ql.Period(n, ql.Years) for n in range(1, 31)]
    yields = [curve.zeroRate(d, day_count, ql.Compounded, ql.Annual).rate() for d in dates]

    # 結果をDataFrameで表示
    df = pd.DataFrame({
        "Date": [d.to_date() for d in dates],
        "Yield": yields
    })
    dfs.append(df)
    date_dates.append(eval_date)

    if eval_date.day == 1 and eval_date.month == 2:
        df.plot(x="Date", y="Yield", title=f"Yield Curve ({eval_date.year}/{eval_date.month}/{eval_date.day})", xlabel="Date", ylabel="Yield (%)")


# アニメーション設定
fig, ax = plt.subplots()
line, = ax.plot([], [], lw=2)  # 空のラインを初期化
ax.set_xlim(0, len(df["Date"]))
ax.set_ylim(min(df["Yield"].min() for df in dfs), max(df["Yield"].max() for df in dfs))

# フレーム更新関数
def update(frame):
    df = dfs[frame]
    line.set_data([i for i in range(0, len(df["Date"]))], df["Yield"])
    ax.set_title(f"{date_dates[frame]}")
    return line,

# アニメーション生成
ani = FuncAnimation(
    fig, 
    update, 
    frames=len(dfs),  # DataFrameの数だけ繰り返す
    blit=True,
    # interval=200  # フレーム間の遅延（ms）
)
# plt.show()
# GIF保存
ani.save("yield_curve_animation.gif", writer=PillowWriter(fps=10))
plt.close()


[*********************100%***********************]  4 of 4 completed


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation, PillowWriter

# データ準備
fig, ax = plt.subplots()
x = np.linspace(0, 2 * np.pi, 100)
line, = ax.plot(x, np.sin(x))

# アニメーションの更新関数
def update(frame):
    line.set_ydata(np.sin(x + frame * 0.1))
    return line,

# アニメーション作成
ani = FuncAnimation(fig, update, frames=50, blit=True)

# GIFとして保存
ani.save("sin_wave.gif", writer=PillowWriter(fps=10))

plt.close()

In [ ]:
# yfinanceでTreasuryイールドを取得
tickers = {
    "^IRX": ql.Period(3, ql.Months),   # 13週T-Bill（約3ヶ月）
    "^FVX": ql.Period(5, ql.Years),    # 5年Treasury Note
    "^TNX": ql.Period(10, ql.Years),   # 10年Treasury Note
    "^TYX": ql.Period(30, ql.Years),   # 30年Treasury Bond
}

# print(date_range)

for ticker, tenor in tickers.items():
    treasury = yf.Ticker(ticker)
print(treasury.history())

                            Open   High    Low  Close  Volume  Dividends  \
Date                                                                       
2025-03-11 00:00:00-05:00  4.557  4.612  4.522  4.603       0        0.0   
2025-03-12 00:00:00-05:00  4.612  4.641  4.573  4.633       0        0.0   
2025-03-13 00:00:00-05:00  4.652  4.671  4.581  4.597       0        0.0   
2025-03-14 00:00:00-05:00  4.639  4.646  4.604  4.615       0        0.0   
2025-03-17 00:00:00-05:00  4.585  4.626  4.556  4.599       0        0.0   
2025-03-18 00:00:00-05:00  4.624  4.641  4.569  4.579       0        0.0   
2025-03-19 00:00:00-05:00  4.589  4.642  4.563  4.568       0        0.0   
2025-03-20 00:00:00-05:00  4.502  4.561  4.490  4.557       0        0.0   
2025-03-21 00:00:00-05:00  4.537  4.599  4.529  4.596       0        0.0   
2025-03-24 00:00:00-05:00  4.639  4.663  4.624  4.655       0        0.0   
2025-03-25 00:00:00-05:00  4.698  4.700  4.639  4.656       0        0.0   
2025-03-26 0